In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras import models, optimizers, callbacks
from tensorflow.keras.layers import (
    Conv1D, LSTM, Dense, Dropout, Input,
    BatchNormalization
)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pickle, os
from datetime import datetime

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

SEQUENCES_DIR = '../data_new/sequences/'
MODELS_DIR = '../models/hybrid/'
RESULTS_DIR = '../results/'
FIGURES_DIR = '../results/figures/hybrid/'

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)

ASSETS = ['AAPL', 'AMZN', 'NVDA', 'SPY', 'BTC-USD']
HORIZONS = ['1day', '1week', '1month']

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

print(f"TensorFlow: {tf.__version__}")
print("[OK] Setup complete")

In [ ]:
# Hybrid CNN-LSTM Model Builder
def build_hybrid_cnn_lstm_model(
    sequence_length, n_features,
    conv_filters=[64, 128],
    kernel_size=3,
    lstm_units=128, lstm_layers=2,
    dropout_rate=0.3, dense_units=64,
    learning_rate=0.001
):
    model = models.Sequential(name='Hybrid_CNN_LSTM_Model')
    model.add(Input(shape=(sequence_length, n_features)))
    
    # CNN layers for feature extraction
    for i, n_filters in enumerate(conv_filters):
        model.add(Conv1D(
            filters=n_filters, kernel_size=kernel_size,
            activation='relu', padding='same', name=f'conv1d_{i+1}'
        ))
        model.add(BatchNormalization(name=f'batch_norm_{i+1}'))
        model.add(Dropout(dropout_rate, name=f'dropout_conv_{i+1}'))
    
    # LSTM layers for temporal modeling
    for i in range(lstm_layers):
        return_sequences = (i < lstm_layers - 1)
        model.add(LSTM(
            units=lstm_units, return_sequences=return_sequences,
            name=f'lstm_{i+1}'
        ))
        model.add(Dropout(dropout_rate, name=f'dropout_lstm_{i+1}'))
    
    # Dense layers
    model.add(Dense(dense_units, activation='relu', name='dense_1'))
    model.add(Dropout(dropout_rate, name='dropout_dense'))
    model.add(Dense(1, activation='sigmoid', name='output'))
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')]
    )
    return model

def load_sequences(asset, horizon):
    filepath = f'{SEQUENCES_DIR}{asset}_{horizon}_sequences.npz'
    data = np.load(filepath)
    return (data['X_train'], data['X_val'], data['X_test'],
            data['y_train'], data['y_val'], data['y_test'],
            int(data['sequence_length']), int(data['n_features']))

def load_class_weights():
    with open(f'{SEQUENCES_DIR}class_weights.pkl', 'rb') as f:
        return pickle.load(f)

def get_callbacks(model_name, patience=10):
    return [
        callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, verbose=1),
        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1),
        callbacks.ModelCheckpoint(filepath=f'{MODELS_DIR}{model_name}_best.h5', monitor='val_loss', save_best_only=True)
    ]

class_weights = load_class_weights()
print("[OK] Functions defined")

In [ ]:
# Complete training loop
all_results = []

print("Starting Hybrid CNN-LSTM training...")
print("="*80)

for asset in ASSETS:
    for horizon in HORIZONS:
        print(f"\n{'='*80}")
        print(f"Training: {asset} - {horizon}")
        start_time = datetime.now()
        
        X_train, X_val, X_test, y_train, y_val, y_test, seq_len, n_feat = load_sequences(asset, horizon)
        
        model = build_hybrid_cnn_lstm_model(
            sequence_length=seq_len, n_features=n_feat,
            conv_filters=[64, 128], lstm_units=128,
            lstm_layers=2, dropout_rate=0.3
        )
        
        cw = class_weights[(asset, horizon)]
        class_weight_dict = {0: cw[0], 1: cw[1]}
        
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=100, batch_size=32,
            class_weight=class_weight_dict,
            callbacks=get_callbacks(f'Hybrid_{asset}_{horizon}'),
            verbose=0
        )
        
        y_pred_proba = model.predict(X_test, verbose=0)
        y_pred = (y_pred_proba > 0.5).astype(int).flatten()
        
        result = {
            'asset': asset, 'horizon': horizon,
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'epochs_trained': len(history.history['loss'])
        }
        all_results.append(result)
        
        elapsed = (datetime.now() - start_time).total_seconds()
        print(f"[OK] Done in {elapsed:.1f}s | Acc: {result['accuracy']:.4f} | F1: {result['f1']:.4f}")

print("\n" + "="*80)
print("[OK] Hybrid CNN-LSTM training complete!")

In [ ]:
# Save and analyze results
hybrid_results = pd.DataFrame(all_results)
hybrid_results.to_csv(f'{RESULTS_DIR}hybrid_results_complete.csv', index=False)

print("\nHybrid CNN-LSTM Model Results:")
print("="*120)
print(hybrid_results.to_string(index=False))

print(f"\n{'='*80}")
print(f"Mean Accuracy: {hybrid_results['accuracy'].mean():.4f} ± {hybrid_results['accuracy'].std():.4f}")

In [ ]:
# Final comparison of ALL 5 models
lstm_results = pd.read_csv(f'{RESULTS_DIR}lstm_results_complete.csv')
gru_results = pd.read_csv(f'{RESULTS_DIR}gru_results_complete.csv')
cnn_results = pd.read_csv(f'{RESULTS_DIR}cnn_results_complete.csv')
transformer_results = pd.read_csv(f'{RESULTS_DIR}transformer_results_complete.csv')

final_comparison = pd.DataFrame({
    'asset': hybrid_results['asset'],
    'horizon': hybrid_results['horizon'],
    'Hybrid': hybrid_results['accuracy'],
    'Transformer': transformer_results['accuracy'],
    'LSTM': lstm_results['accuracy'],
    'GRU': gru_results['accuracy'],
    'CNN': cnn_results['accuracy']
})

final_comparison['best_model'] = final_comparison[['Hybrid', 'Transformer', 'LSTM', 'GRU', 'CNN']].idxmax(axis=1)
final_comparison['best_accuracy'] = final_comparison[['Hybrid', 'Transformer', 'LSTM', 'GRU', 'CNN']].max(axis=1)

print("\n" + "="*120)
print("FINAL MODEL COMPARISON - ALL 5 ARCHITECTURES")
print("="*120)
print(final_comparison.to_string(index=False))

print(f"\n{'='*80}")
print("Overall Performance by Model:")
print("="*80)
for model in ['Hybrid', 'Transformer', 'LSTM', 'GRU', 'CNN']:
    mean_acc = final_comparison[model].mean()
    std_acc = final_comparison[model].std()
    wins = (final_comparison['best_model'] == model).sum()
    print(f"{model:12s}: {mean_acc:.4f} ± {std_acc:.4f} | Wins: {wins}/16")

# Save final comparison
final_comparison.to_csv(f'{RESULTS_DIR}all_models_final_comparison.csv', index=False)
print(f"\n[OK] Final comparison saved")

In [ ]:
# Visualize: Bar chart comparing all models
fig, ax = plt.subplots(figsize=(14, 8))

model_names = ['Hybrid', 'Transformer', 'LSTM', 'GRU', 'CNN']
means = [final_comparison[m].mean() for m in model_names]
stds = [final_comparison[m].std() for m in model_names]

x = np.arange(len(model_names))
bars = ax.bar(x, means, yerr=stds, alpha=0.8, capsize=5, edgecolor='black', linewidth=1.5)

# Color bars
colors = ['#9b59b6', '#e74c3c', '#3498db', '#2ecc71', '#f39c12']
for bar, color in zip(bars, colors):
    bar.set_color(color)

ax.set_ylabel('Mean Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Final Model Comparison - Mean Accuracy Across All Assets and Horizons', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(model_names, fontsize=12)
ax.grid(True, alpha=0.3, axis='y')
ax.axhline(y=0.5, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Random Baseline')
ax.legend()

# Add value labels on bars
for i, (m, s) in enumerate(zip(means, stds)):
    ax.text(i, m + s + 0.005, f'{m:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig(f'{FIGURES_DIR}all_models_final_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("[OK] Final comparison visualization saved")

## Summary: All Deep Learning Models

**Architecture Training Complete**:
- [OK] LSTM: Good temporal modeling, standard choice
- [OK] GRU: Efficient alternative to LSTM
- [OK] CNN: Fast, good on short horizons
- [OK] Transformer: Best for long-range dependencies
- [OK] Hybrid CNN-LSTM: Balanced multi-scale approach

**Next Steps**:
-> **Notebook 11**: Hyperparameter tuning for top models  
-> **Notebook 12**: Cross-asset transfer learning  
-> **Notebook 13**: Financial backtesting  
-> **Notebook 14**: Model interpretation  
-> **Notebook 15**: Final report

---
[OK] **All model architectures trained!**